In [1]:
import csv
import json
import unicodedata
import numpy as np
import networkx as nx
import requests
from pattern import web
import matplotlib.pyplot as plt

# set some nicer defaults for matplotlib
from matplotlib import rcParams

def get_poll_xml1(poll_id):
    url = "http://www.nhl.com/scores/htmlreports/20142015/GS0%i.HTM" % int(poll_id)
    return requests.get(url).text

from bs4 import BeautifulSoup
Scorers=[]
Scorers3=[]
FstAssistsPlayers=[]
SndAssistsPlayers=[]
plusMinusPlayers=[]
# [20036,20461,20846,21114,21151,21164,21176]:
# [20036,20149,20235,20332,20461,20742,20846,21114,21151,21164,21176]:
# [20149,20332,20742]
listi=[20036,20078,20149,20232,20235,20290,20332,20441,20461,20544,20547,20742,20755,20812,20846,20903,21025,21114,21144,21151,21164,21176]

# [20036,20078,20232,20235,20290,20441,20461,20755,20812,20846,20903,21025,21114,21144,21151,21164,21176]
for numGames in range(20000,21231):
    if numGames not in [20036,20078,20149,20232,20235,20290,20332,20441,20461,20544,20547,20742,20755,20812,20846,20903,21025,21114,21144,21151,21164,21176]:
        print numGames
        soup = BeautifulSoup(get_poll_xml1(numGames))
        # soup

        result1= soup.find_all("tr", attrs={"class":"oddColor"} )
#         resultCenter= soup.find_all("td", attrs={"align":"center","class":"oddColor"} )
#         resultCen= result1.find_all("td", attrs={"align":"center"} )
        resultCen=[elem.find_all("td", attrs={"align":"center"} ) for elem in result1]
#         print resultCen
        
        result2=[elem.find_all("td", attrs={"align":"left"} ) for elem in result1]

        resultEven1= soup.find_all("tr", attrs={"class":"evenColor"} )

        resultEven=[elem.find_all("td", attrs={"align":"left"} ) for elem in resultEven1]

        result3= soup.find_all("td", attrs={"align":"center", "style":"font-size: 40px;font-weight:bold"} )

        totalGoals=sum([int(elem.text.strip()) for elem in result3])
        resultCenEven=[elem.find_all("td", attrs={"align":"center"} ) for elem in resultEven1]
        if totalGoals%2==0:
            ODDGoals=totalGoals/2
            EVENGoals=totalGoals/2
        else:
            ODDGoals=totalGoals/2+1
            EVENGoals=totalGoals/2

        oddGoals=[]
        oddFstAssists=[]
        oddSndAssists=[]
        oddarrayFailure=[]
        for x in range(0,ODDGoals):
            if len(result2[x])==3:
                oddGoals.append(result2[x][0])
                oddFstAssists.append(result2[x][1])
                oddSndAssists.append(result2[x][2])
            else:
                oddarrayFailure.append(x)

        evenGoals=[]
        evenFstAssists=[]
        evenSndAssists=[]    
        evenarrayFailure=[]
        for x in range(0,EVENGoals):
        #     print "Test"
        #     print 
            if len(resultEven[x])==3:
                evenGoals.append(resultEven[x][0])
        #         print resultEven[x][1]
                evenFstAssists.append(resultEven[x][1])
                evenSndAssists.append(resultEven[x][2])
            else:
                evenarrayFailure.append(x)
        #     else:
        #         evenGoals.append(resultEven[x][0])
        # #         print resultEven[x][1]
        #         evenFstAssists.append(resultEven[x][1])
        #         evenSndAssists.append()



        resultplayers=soup.find_all("font", attrs={"style":"cursor:hand;"} )

        tds=soup.find_all("td" )[1:]
        tds=[str(elem) for elem in tds]

        count=[]
        tds2=[]
    #     print tds

        for x in range(0,len(tds)):
            if "font style=" in tds[x]:
                count.append(tds[x].count ("font style="))
                tds2.append(tds[x])
    #     print tds2[2]
    #     print tds2
        if tds2!=[]:
            indices = [i for i, x in enumerate(tds2[2]) if x == ","]
        #     print tds2[2]
            temp=[]
            for x in range(0,len(indices)):
                if x==0:
                    temp.append(tds2[2][:indices[0]])
                    temp.append(tds2[2][indices[x]:indices[x+1]])
                elif x!=len(indices)-1:
                    temp.append(tds2[2][indices[x]:indices[x+1]])
                elif x==len(indices)-1:
                    temp.append(tds2[2][indices[x]:len(tds2[2])])
    #         print temp
            if temp!=[]:
                temp[0]=temp[0].replace("<td>", "")
        #     print temp

        #     print temp
            for x in range(0,len(temp)):
                temp[x]=temp[x][(temp[x].index(">")+1):len(temp[x])]
                temp[x]=temp[x][0:(temp[x].index("<"))] 
            arrayTemp=[]

            indices = []

            arrayTemp=[]
            for y in range(1,len(tds2)):
                temp=[]
                indices = [i for i, x in enumerate(tds2[y]) if x == ","]
            #     print tds2[y]
            #     print indices
                for x in range(0,len(indices)):
                    if x==0:
                        temp.append(tds2[y][:indices[0]])
                        temp.append(tds2[y][indices[x]:indices[x+1]])
                    elif x!=len(indices)-1:
                        temp.append(tds2[y][indices[x]:indices[x+1]])
                    elif x==len(indices)-1:
                        temp.append(tds2[y][indices[x]:len(tds2[y])])
        #         print y
        #         print temp
                if temp!=[]:
                    temp[0]=temp[0].replace("<td>", "")
            #     print y
            #     print temp
                for x in range(0,len(temp)):
                    temp[x]=temp[x][(temp[x].index(">")+1):len(temp[x])]
                    temp[x]=temp[x][0:(temp[x].index("<"))]
                arrayTemp.append(temp)



            resultTeam=[elem.find_all("td", attrs={"align":"left","class":"heading + bborder"} ) for elem in soup]
            homeTeam=resultTeam[0][3].text.strip()[:resultTeam[0][3].text.strip().index(" ")]
            awayTeam=resultTeam[0][4].text.strip()[:resultTeam[0][4].text.strip().index(" ")]
            resultTeam1= soup.find_all("tr", attrs={"class":"oddColor"} )
            resultTeam2=[elem.find_all("td", attrs={"align":"center"} ) for elem in resultTeam1]

            result1TeamEven= soup.find_all("tr", attrs={"class":"evenColor"} )
            result2TeamEven=[elem.find_all("td", attrs={"align":"center"} ) for elem in result1TeamEven]
            oddGoalTeam=[]
            evenGoalTeam=[]
        #     print "EVENGoals"
        #     print ODDGoals
        #     print EVENGoals
        #     print arrayTemp
#             print "evenarrayFailure"
#             print evenarrayFailure
        #     # oddFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddFstAssists]
        #     # print indicesloc

        #     print arrayTemp
    #         print 
#             print result2TeamEven
#             print "result2TeamEven-"
#             print result2TeamEven
#             print result2TeamEven[0][4]
#             print result2TeamEven[1][4]
#             print result2TeamEven[2][4]
#             print result2TeamEven[3][4]
            for x in range(0,ODDGoals):
                if x  not in oddarrayFailure:
                    oddGoalTeam.append(str(resultTeam2[x][4].text.strip()))
#             print evenarrayFailure
#             print EVENGoals
            for x in range(0,EVENGoals):
        #         print x
                if x not in evenarrayFailure:
#                     print x
                    evenGoalTeam.append(str(result2TeamEven[x][4].text.strip()))

            indicesloc = [i for i, x in enumerate(arrayTemp) if x == []]
            # print arrayTemp.index([])
        #     print indicesloc 
            # indicesloc=[]
            indicesloc= [ elem/2  for elem in indicesloc if elem%2==0]
        #     print "result2TeamEven[indicesloc[0]]"
        #     print indicesloc
        #     print result2TeamEven[0]
        #     print result2TeamEven[1]
        #     print result2TeamEven[2]
        #     print result2TeamEven[3]
        #     print result2TeamEven[4]
        #     print result2TeamEven[5]
        #     print "result2TeamEven[indicesloc[0]/2]"
            # print result2TeamEven[indicesloc[0]/2]

            # print result2TeamEven
    #         print indicesloc
    #         print totalGoals
            # print result2TeamEven[indicesloc[0]]
    #         print evenGoalTeam
#             print "result2[0]"
#             print result2[0]
#             print result2[1]
#             print result2[2]
            
            totalGoals1=totalGoals
            for x in range(0,len(indicesloc)):
    #             print x
    #             if totalGoals%2==0 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
                totalGoals1=totalGoals1+1
#             for x in range(0,len(indicesloc)):
#                 if totalGoals1%2==0:
# #                     print "Even!"
#                     oddGoalTeam.append((str(resultTeam2[indicesloc[x]/2][4].text.strip())))
#     #                 evenGoalTeam.append((str(result2TeamEven[indicesloc[x]/2][4].text.strip())))
#     #             elif totalGoals%2==1 and (indicesloc[x]!=totalGoals  and indicesloc[x]!=totalGoals-1):
#                 elif totalGoals1%2==1:
# #                     print "Odd!"
#     #                 oddGoalTeam.append((str(resultTeam2[indicesloc[x]/2][4].text.strip())))
#                     evenGoalTeam.append((str(result2TeamEven[indicesloc[x]/2][4].text.strip())))
            lenOddGoalTeam=len(oddGoalTeam)
            lenEvenGoalTeam=len(evenGoalTeam)
            for x in range(0,len(indicesloc)):
                if totalGoals1%2==1:
#                     print "Even!"
                    oddGoalTeam.append((str(resultTeam2[lenOddGoalTeam][4].text.strip())))
    #                 evenGoalTeam.append((str(result2TeamEven[indicesloc[x]/2][4].text.strip())))
    #             elif totalGoals%2==1 and (indicesloc[x]!=totalGoals  and indicesloc[x]!=totalGoals-1):
                elif totalGoals1%2==0:
#                     print "Odd!"
    #                 oddGoalTeam.append((str(resultTeam2[indicesloc[x]/2][4].text.strip())))
                    evenGoalTeam.append((str(result2TeamEven[lenEvenGoalTeam][4].text.strip())))
    #         print indicesloc
    #         print totalGoals
    #         print evenGoalTeam
#             print result2TeamEven
#             print indicesloc
#             print "str(result2TeamEven[indicesloc[x]/2][4].text.strip())-"
#             print str(result2TeamEven[indicesloc[x]/2][4].text.strip())
            if totalGoals%2==0:
                ODDGoals1=totalGoals1/2
                EVENGoals1=totalGoals1/2
            else:
                ODDGoals1=totalGoals1/2+1
                EVENGoals1=totalGoals1/2
#             print "resultCen[totalGoals1]"
#             print resultCen[EVENGoals1][2]
#             print resultCen[ODDGoals1][2]
#             print resultCen[EVENGoals1][1]
#             print resultCen[ODDGoals1][1]
#             print resultCen[EVENGoals1][0]
            resultCenEVENGoals1=str(resultCen[EVENGoals1][1])
#             print resultCenEVENGoals1[(resultCenEVENGoals1.index(">")+1):len(resultCenEVENGoals1)]
#             print str(resultCen[EVENGoals1][1])
#             print str(resultCen[EVENGoals1][1])[(str(resultCen[EVENGoals1][1]).index(">")+1):]
#             print "resultCen-"
#             print resultCen
#             print "resultCenEven-"
#             print resultCenEven
#             print "totalGoals1-8"
#             print totalGoals1
#             print totalGoals
            if totalGoals1==totalGoals:
                if totalGoals1%2==0:
#                     print "strtemp-1"
                    strtemp=str(resultCenEven[EVENGoals1-1][1])[(str(resultCenEven[EVENGoals1-1][1]).index(">"))+1:]
                    strtemp=strtemp[0:(strtemp.index("<"))] 
                    manPower=str(resultCenEven[EVENGoals1-1][3])[(str(resultCenEven[EVENGoals1-1][3]).index(">"))+1:]
                    manPower=manPower[0:(manPower.index("<"))] 
                else:
#                     print "strtemp-2"
                    strtemp=str(resultCen[ODDGoals1-1][1])[(str(resultCen[ODDGoals1-1][1]).index(">"))+1:]
                    strtemp=strtemp[0:(strtemp.index("<"))]
                    manPower=str(resultCen[ODDGoals1-1][3])[(str(resultCen[ODDGoals1-1][3]).index(">"))+1:]
                    manPower=manPower[0:(manPower.index("<"))] 
            else:
                if totalGoals1%2==0:
                    strtemp=str(resultCenEven[EVENGoals1-1][1])[(str(resultCenEven[EVENGoals1-1][1]).index(">"))+1:]
                    strtemp=strtemp[0:(strtemp.index("<"))] 
                    manPower=str(resultCenEven[EVENGoals1-1][3])[(str(resultCenEven[EVENGoals1-1][3]).index(">"))+1:]
                    manPower=manPower[0:(manPower.index("<"))] 
                else:
                    strtemp=str(resultCen[ODDGoals1-1][1])[(str(resultCen[ODDGoals1-1][1]).index(">"))+1:]
                    strtemp=strtemp[0:(strtemp.index("<"))]
                    manPower=str(resultCen[ODDGoals1-1][3])[(str(resultCen[ODDGoals1-1][3]).index(">"))+1:]
                    manPower=manPower[0:(manPower.index("<"))] 
            
#             print resultCen[ODDGoals1][3]
            
#         <td align="center">SO</td>
#             temp[x]=temp[x][(temp[x].index(">")+1):len(temp[x])]
#                 temp[x]=temp[x][0:(temp[x].index("<"))] 

#             print resultCen[ODDGoals1][0]
            
                
    #         [1]
    # 3
    # []%%!
#             print numGames
#             print "manPower--"
#             print manPower
#             print totalGoals1
#             print "EVENGoals1"
#             print EVENGoals1
#             print "ODDGoals1"
#             print ODDGoals1
#             print resultCenEven
            
#             if manPower=="EV":
            oddArrayTemp=[]
            evenArrayTemp=[]
            for y in range(0,len(arrayTemp)):
                if y%4==1:
                    oddArrayTemp.append(arrayTemp[y]) 
                elif y%4==0:
                    oddArrayTemp.append(arrayTemp[y]) 
                elif y%4==2:
                    evenArrayTemp.append(arrayTemp[y]) 
                elif y%4==3:
                    evenArrayTemp.append(arrayTemp[y])
        #     print "oddArrayTemp"   
            # print arrayTemp.index([])

        #     print oddGoalTeam
        #     print evenGoalTeam
        #     print oddArrayTemp
        #     for y in range(0,len(oddArrayTemp)/2):
        #         if oddGoalTeam[y]==homeTeam:
        #             oddArrayTemp[2*y].append("W")
        #             oddArrayTemp[2*y+1].append("L")
        #         else:
        #             oddArrayTemp[2*y].append("L")
        #             oddArrayTemp[2*y+1].append("W")

        #     print evenArrayTemp
        #     print evenGoalTeam
        #     print evenarrayFailure
        #     print len(evenArrayTemp)
        #     print arrayTemp
        #     print len(evenArrayTemp)/2
        #     print len(evenGoalTeam)
        #     print evenGoalTeam
            
            evenArrayTemp=filter(lambda a: a != [], evenArrayTemp)
            oddArrayTemp=filter(lambda a: a != [], oddArrayTemp)
            # evenArrayTemp.remove([])
        #     print evenArrayTemp
#             print oddGoalTeam
    #         print oddArrayTemp
#             print "evenGoalTeam"
#             print evenGoalTeam
    #         print evenArrayTemp
            for y in range(0,len(oddArrayTemp)/2):
                if oddGoalTeam[y]==homeTeam:
                    oddArrayTemp[2*y].insert(0,"W")
                    oddArrayTemp[2*y].insert(0,homeTeam)
                    oddArrayTemp[2*y+1].insert(0,"L")
                    oddArrayTemp[2*y+1].insert(0,awayTeam)
                else:
                    oddArrayTemp[2*y].insert(0,"L")
                    oddArrayTemp[2*y].insert(0,homeTeam)
                    oddArrayTemp[2*y+1].insert(0,"W")
                    oddArrayTemp[2*y+1].insert(0,awayTeam)
    #         print evenArrayTemp
    #         print evenGoalTeam
#             print 
            for y in range(0,len(evenArrayTemp)/2):
                if evenGoalTeam[y]==homeTeam:
                    evenArrayTemp[2*y].insert(0,"W")
                    evenArrayTemp[2*y].insert(0,homeTeam)
                    evenArrayTemp[2*y+1].insert(0,"L")
                    evenArrayTemp[2*y+1].insert(0,awayTeam)
                else:
                    evenArrayTemp[2*y].insert(0,"L")
                    evenArrayTemp[2*y].insert(0,homeTeam)
                    evenArrayTemp[2*y+1].insert(0,"W")
                    evenArrayTemp[2*y+1].insert(0,awayTeam)

        #     for y in range(0,len(evenArrayTemp)/2):
        #     #     print y
        #         if evenGoalTeam[y]==homeTeam:
        #             evenArrayTemp[2*y].append("W")
        #             evenArrayTemp[2*y+1].append("L")
        #         else:
        #             evenArrayTemp[2*y].append("L")
        #             evenArrayTemp[2*y+1].append("W")

        #     print 
        #     print result2[1]
        #     print result2[2]
        #     print result2[3]
        #     print indicesloc
    #         print totalGoals
    #         print indicesloc
            for x in range(0,len(indicesloc)):
            #     print x
    #             if totalGoals%2==0 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
                if totalGoals%2==0:
            #         evenGoalTeam.append((str(result2TeamEven[indicesloc[x]][4].text.strip())))
    #                 evenGoals.append(result2[EVENGoals][0])
    #                 evenFstAssists.append(result2[EVENGoals][1])
    #                 evenSndAssists.append(result2[EVENGoals][2])
                    if len(result2[ODDGoals])==3:
                        oddGoals.append(result2[ODDGoals][0])
                        oddFstAssists.append(result2[ODDGoals][1])
                        oddSndAssists.append(result2[ODDGoals][2])
    #             elif totalGoals%2==1 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
                elif totalGoals%2==1:
            #         oddGoalTeam.append((str(resultTeam2[indicesloc[x]][4].text.strip())))
    #                 oddGoals.append(result2[ODDGoals][0])
    #                 oddFstAssists.append(result2[ODDGoals][1])
    #                 oddSndAssists.append(result2[ODDGoals][2])
#                     print resultEven
                    if len(resultEven[EVENGoals])==3:
                        evenGoals.append(resultEven[EVENGoals][0])
                        evenFstAssists.append(resultEven[EVENGoals][1])
                        evenSndAssists.append(resultEven[EVENGoals][2])

            # oddGoals.append(result2[ODDGoals][0])
            # oddFstAssists.append(result2[ODDGoals][1])
            # oddSndAssists.append(result2[ODDGoals][2])
            
            oddGoalsNumber= [str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddGoals]
            oddFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddFstAssists]
            oddSndNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddSndAssists]
            evenGoalsNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenGoals]
            evenFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenFstAssists]
            evenSndNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenSndAssists ]
        #     print "evenGoalsNumber"
        #     print oddGoals
            # for x in range(0,len(indicesloc)):
            #     print x
            #     if indicesloc[x]%2==0:
            #         evenGoalsNumber.append((str(result2TeamEven[indicesloc[x]][4].text.strip())))
            #     else:
            #         oddGoalsNumber.append((str(resultTeam2[indicesloc[x]][4].text.strip())))

            goalOdd = []
            fstAssistOdd = []
            sndAssistOdd = []

            for y in range(0,len(oddGoalTeam)):
                goalOdd.append([])
                fstAssistOdd.append([])
                sndAssistOdd.append([])
        #     print oddGoalsNumber
        #     print oddFstNumber
        #     print oddSndNumber
        #     print oddGoalTeam
            for y in range(0,len(oddGoalTeam)):
                goalOdd[y].append(oddGoalsNumber[y])
                fstAssistOdd[y].append(oddFstNumber[y])
                sndAssistOdd[y].append(oddSndNumber[y])

            goalEven = []
            fstAssistEven = []
            sndAssistEven = []

            for y in range(0,len(evenGoalTeam)):
                goalEven.append([])
                fstAssistEven.append([])
                sndAssistEven.append([])

#             print evenGoalsNumber
        #     print evenFstNumber
        #     print evenSndNumber
#             print evenGoalTeam
        #     print evenarrayFailure

            for y in range(0,len(evenGoalTeam)):
                goalEven[y].append(evenGoalsNumber[y])
                fstAssistEven[y].append(evenFstNumber[y])
                sndAssistEven[y].append(evenSndNumber[y])


            for y in range(0,len(goalEven)):
                if evenGoalTeam[y]==homeTeam:
                    goalEven[y].insert(0,str(homeTeam))
                    fstAssistEven[y].insert(0,str(homeTeam))
                    sndAssistEven[y].insert(0,str(homeTeam))
                else:
                    goalEven[y].insert(0,str(awayTeam))
                    fstAssistEven[y].insert(0,str(awayTeam))
                    sndAssistEven[y].insert(0,str(awayTeam))

        #     print "goalEven"
        #     print goalEven
        #     print fstAssistEven
        #     print sndAssistEven
    #         print oddGoalTeam
    #         print goalOdd
            for y in range(0,len(goalOdd)):
                if oddGoalTeam[y]==homeTeam:
                    goalOdd[y].insert(0,str(homeTeam))
                    fstAssistOdd[y].insert(0,str(homeTeam))
                    sndAssistOdd[y].insert(0,str(homeTeam))
                else:
                    goalOdd[y].insert(0,str(awayTeam))
                    fstAssistOdd[y].insert(0,str(awayTeam))
                    sndAssistOdd[y].insert(0,str(awayTeam))

        #     print "goalOdd"
        #     print goalOdd
        #     print fstAssistOdd
        #     print sndAssistOdd



        #     print goalEven+goalOdd
        #     print fstAssistEven+fstAssistOdd
        #     print sndAssistEven+sndAssistOdd
#             print "evenArrayTemp+oddArrayTemp"
#             print evenArrayTemp+oddArrayTemp
            listEvenManPower=[]
            listOddManPower=[]
            for num in range(0,EVENGoals1):
#                 print "resultCenEven[0]"
#                 print resultCenEven[num][3]
                listEvenManPower.append(resultCenEven[num][3].string)
            for num in range(0,ODDGoals1):
#                 print "resultCen[0]"
#                 print resultCen[num][3]
                listOddManPower.append(resultCen[num][3].string)
#             print "listEvenManPower--"
#             print listEvenManPower
#             print "listOddManPower--"
#             print listOddManPower
            listEvenManPower=filter(lambda a: a != None, listEvenManPower)
            listOddManPower=filter(lambda a: a != None, listOddManPower)
            listOddManPower=filter(lambda a: str(a).isdigit() == False, listOddManPower)
            listEvenManPower=filter(lambda a: str(a).isdigit() == False, listEvenManPower)
#             print "listEvenManPower1--"
#             print listEvenManPower
#             print "listOddManPower--"
#             print listOddManPower
            listOddManPower1=[]
            for elem in listOddManPower:
                if "-PS" not in str(elem):
                    listOddManPower1.append(elem)
            listOddManPower=listOddManPower1
            listEvenManPower1=[]
            for elem in listEvenManPower:
                if "-PS" not in str(elem):
                    listEvenManPower1.append(elem)
            listEvenManPower=listEvenManPower1
#             listOddManPower=filter(lambda a: "-PS" in str(a) == False, listOddManPower)
            
#             listEvenManPower=filter(lambda a: "-PS" in str(a) == False, listEvenManPower)
#             print "listEvenManPower2--"
#             print listEvenManPower
#             print "listOddManPower2--"
#             print listOddManPower
            goalEvenPP=[]
            fstAssistEvenPP=[]
            sndAssistEvenPP=[]
            evenArrayTempPP=[]
            for i in range(0,len(listEvenManPower)):
                if listEvenManPower[i]=="PP":
                    goalEvenPP.append(goalEven[i])
                    fstAssistEvenPP.append(fstAssistEven[i])
                    sndAssistEvenPP.append(sndAssistEven[i])
                    evenArrayTempPP.append(evenArrayTemp[2*i])
                    evenArrayTempPP.append(evenArrayTemp[2*i+1])
            goalOddPP=[]
            fstAssistOddPP=[]
            sndAssistOddPP=[]
            oddArrayTempPP=[]
#             print listOddManPower
            
#             print goalOdd
            for i in range(0,len(listOddManPower)):
                if listOddManPower[i]=="PP":
                    goalOddPP.append(goalOdd[i])   
                    fstAssistOddPP.append(fstAssistOdd[i])
                    sndAssistOddPP.append(sndAssistOdd[i])
                    oddArrayTempPP.append(oddArrayTemp[2*i])
                    oddArrayTempPP.append(oddArrayTemp[2*i+1])
#             print "goalEvenPP-"
#             print goalEvenPP
#             print "temp----"
#             print temp
            
            GoalPP=goalEvenPP+goalOddPP
            FstAssistPP=fstAssistEvenPP+fstAssistOddPP
            SndAssistPP=sndAssistEvenPP+sndAssistOddPP
            ArrayTempPP=evenArrayTempPP+oddArrayTempPP
            Goal=goalEven+goalOdd
            FstAssist=fstAssistEven+fstAssistOdd
            SndAssist=sndAssistEven+sndAssistOdd
            ArrayTemp=evenArrayTemp+oddArrayTemp
#             print "strtemp"
#             print strtemp
#             print manPower
            if manPower=="SH-EN":
                del ArrayTemp[-2]
                del FstAssist[-1]
                del SndAssist[-1]
                del Goal[-1]
#             if manPower=="EV-EN":
#                 del ArrayTemp[-1]
#                 del FstAssist[-1]
#                 del SndAssist[-1]
#                 del Goal[-1]
            if strtemp=="SO":
#                 del ArrayTemp[-1]
                del FstAssist[-1]
                del SndAssist[-1]
                del Goal[-1]
#             print "ArrayTemp--"
#             print ArrayTemp
#             print "ArrayTempPP--"
#             print ArrayTempPP
#             print "FstAssist--"
#             print FstAssist
#             print "SndAssist--"
#             print SndAssist
            
#             print "Goal--"
#             print Goal
#             if strtemp=="OT":
#                 del ArrayTemp[-1]
#                 del FstAssist[-1]
#                 del SndAssist[-1]
#                 del Goal[-1]
#             print "strtemp-"
#             print strtemp
#             print "manpower-"
#             print manPower
#             print "oddGoalTeam-"
#             print oddGoalTeam
#             print "evenGoalTeam-"
#             print evenGoalTeam



#             print ArrayTemp
            Scorers.append(GoalPP)
            Scorers3.append(GoalPP)
#             print GoalPP
#             print GoalPP
            FstAssistsPlayers.append(FstAssistPP)
            SndAssistsPlayers.append(SndAssistPP)
            plusMinusPlayers.append(ArrayTempPP)
#     elif numGames in [20149,20235,20332,20742]:
#     elif numGames in [20149,20332,20742]:
    elif numGames in [20149,20742]:
        # [20036,20149,20235,20332,20461,20742,20846,21114,21151,21164,21176]:
#             [20149,20235,20332,20742]:
        print numGames
        soup = BeautifulSoup(get_poll_xml1(numGames))
        # soup

        result1= soup.find_all("tr", attrs={"class":"oddColor"} )

        result2=[elem.find_all("td", attrs={"align":"left"} ) for elem in result1]

        resultEven= soup.find_all("tr", attrs={"class":"evenColor"} )

        resultEven=[elem.find_all("td", attrs={"align":"left"} ) for elem in resultEven]

        result3= soup.find_all("td", attrs={"align":"center", "style":"font-size: 40px;font-weight:bold"} )

        totalGoals=sum([int(elem.text.strip()) for elem in result3])

        if totalGoals%2==0:
            ODDGoals=totalGoals/2
            EVENGoals=totalGoals/2
        else:
            ODDGoals=totalGoals/2+1
            EVENGoals=totalGoals/2

        oddGoals=[]
        oddFstAssists=[]
        oddSndAssists=[]
        oddarrayFailure=[]
        for x in range(0,ODDGoals):
            if len(result2[x])==3:
                oddGoals.append(result2[x][0])
                oddFstAssists.append(result2[x][1])
                oddSndAssists.append(result2[x][2])
            else:
                oddarrayFailure.append(x)

        evenGoals=[]
        evenFstAssists=[]
        evenSndAssists=[]    
        evenarrayFailure=[]
        for x in range(0,EVENGoals):
        #     print "Test"
        #     print 
            if len(resultEven[x])==3:
                evenGoals.append(resultEven[x][0])
        #         print resultEven[x][1]
                evenFstAssists.append(resultEven[x][1])
                evenSndAssists.append(resultEven[x][2])
            else:
                evenarrayFailure.append(x)
        #     else:
        #         evenGoals.append(resultEven[x][0])
        # #         print resultEven[x][1]
        #         evenFstAssists.append(resultEven[x][1])
        #         evenSndAssists.append()



        resultplayers=soup.find_all("font", attrs={"style":"cursor:hand;"} )

        tds=soup.find_all("td" )[1:]
        tds=[str(elem) for elem in tds]

        count=[]
        tds2=[]
    #     print tds

        for x in range(0,len(tds)):
            if "font style=" in tds[x]:
                count.append(tds[x].count ("font style="))
                tds2.append(tds[x])
    #     print tds2[2]
    #     print tds2
        if tds2!=[]:
            indices = [i for i, x in enumerate(tds2[2]) if x == ","]
        #     print tds2[2]
            temp=[]
            for x in range(0,len(indices)):
                if x==0:
                    temp.append(tds2[2][:indices[0]])
                    temp.append(tds2[2][indices[x]:indices[x+1]])
                elif x!=len(indices)-1:
                    temp.append(tds2[2][indices[x]:indices[x+1]])
                elif x==len(indices)-1:
                    temp.append(tds2[2][indices[x]:len(tds2[2])])
    #         print temp
            if temp!=[]:
                temp[0]=temp[0].replace("<td>", "")
        #     print temp

        #     print temp
            for x in range(0,len(temp)):
                temp[x]=temp[x][(temp[x].index(">")+1):len(temp[x])]
                temp[x]=temp[x][0:(temp[x].index("<"))] 
            arrayTemp=[]

            indices = []

            arrayTemp=[]
            for y in range(1,len(tds2)):
                temp=[]
                indices = [i for i, x in enumerate(tds2[y]) if x == ","]
            #     print tds2[y]
            #     print indices
                for x in range(0,len(indices)):
                    if x==0:
                        temp.append(tds2[y][:indices[0]])
                        temp.append(tds2[y][indices[x]:indices[x+1]])
                    elif x!=len(indices)-1:
                        temp.append(tds2[y][indices[x]:indices[x+1]])
                    elif x==len(indices)-1:
                        temp.append(tds2[y][indices[x]:len(tds2[y])])
        #         print y
        #         print temp
                if temp!=[]:
                    temp[0]=temp[0].replace("<td>", "")
            #     print y
            #     print temp
                for x in range(0,len(temp)):
                    temp[x]=temp[x][(temp[x].index(">")+1):len(temp[x])]
                    temp[x]=temp[x][0:(temp[x].index("<"))]
                arrayTemp.append(temp)



            resultTeam=[elem.find_all("td", attrs={"align":"left","class":"heading + bborder"} ) for elem in soup]
            homeTeam=resultTeam[0][3].text.strip()[:resultTeam[0][3].text.strip().index(" ")]
            awayTeam=resultTeam[0][4].text.strip()[:resultTeam[0][4].text.strip().index(" ")]
            resultTeam1= soup.find_all("tr", attrs={"class":"oddColor"} )
            resultTeam2=[elem.find_all("td", attrs={"align":"center"} ) for elem in resultTeam1]

            result1TeamEven= soup.find_all("tr", attrs={"class":"evenColor"} )
            result2TeamEven=[elem.find_all("td", attrs={"align":"center"} ) for elem in result1TeamEven]
            oddGoalTeam=[]
            evenGoalTeam=[]
        #     print "EVENGoals"
        #     print ODDGoals
        #     print EVENGoals
        #     print arrayTemp
        #     print "evenarrayFailure"
        #     print evenarrayFailure
        #     # oddFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddFstAssists]
        #     # print indicesloc

        #     print arrayTemp
    #         print 
            for x in range(0,ODDGoals):
                if x  not in oddarrayFailure:
                    oddGoalTeam.append(str(resultTeam2[x][4].text.strip()))
#             print evenarrayFailure
            for x in range(0,EVENGoals):
        #         print x
                if x not in evenarrayFailure:
        #             print x
                    evenGoalTeam.append(str(result2TeamEven[x][4].text.strip()))

            indicesloc = [i for i, x in enumerate(arrayTemp) if x == []]
            # print arrayTemp.index([])
        #     print indicesloc 
            # indicesloc=[]
            indicesloc= [ elem/2  for elem in indicesloc if elem%2==0]
        #     print "result2TeamEven[indicesloc[0]]"
        #     print indicesloc
        #     print result2TeamEven[0]
        #     print result2TeamEven[1]
        #     print result2TeamEven[2]
        #     print result2TeamEven[3]
        #     print result2TeamEven[4]
        #     print result2TeamEven[5]
        #     print "result2TeamEven[indicesloc[0]/2]"
            # print result2TeamEven[indicesloc[0]/2]

            # print result2TeamEven
    #         print indicesloc
    #         print totalGoals
            # print result2TeamEven[indicesloc[0]]
    #         print evenGoalTeam
            for x in range(0,len(indicesloc)):
    #             print x
    #             if totalGoals%2==0 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
                if totalGoals%2==0 and (indicesloc[x]!=totalGoals or indicesloc[x]!=totalGoals-1):
#                     print "Even!"
                    oddGoalTeam.append((str(resultTeam2[indicesloc[x]/2][4].text.strip())))
    #                 evenGoalTeam.append((str(result2TeamEven[indicesloc[x]/2][4].text.strip())))
    #             elif totalGoals%2==1 and (indicesloc[x]!=totalGoals  and indicesloc[x]!=totalGoals-1):
                elif totalGoals%2==1 and (indicesloc[x]!=totalGoals or indicesloc[x]!=totalGoals-1):
#                     print "Odd!"
    #                 oddGoalTeam.append((str(resultTeam2[indicesloc[x]/2][4].text.strip())))
                    evenGoalTeam.append((str(result2TeamEven[indicesloc[x]/2][4].text.strip())))
    #         print indicesloc
    #         print totalGoals
    #         print evenGoalTeam
    #         [1]
    # 3
    # []
            oddArrayTemp=[]
            evenArrayTemp=[]
            for y in range(0,len(arrayTemp)):
                if y%4==1:
                    oddArrayTemp.append(arrayTemp[y]) 
                elif y%4==0:
                    oddArrayTemp.append(arrayTemp[y]) 
                elif y%4==2:
                    evenArrayTemp.append(arrayTemp[y]) 
                elif y%4==3:
                    evenArrayTemp.append(arrayTemp[y])
        #     print "oddArrayTemp"   
            # print arrayTemp.index([])

        #     print oddGoalTeam
        #     print evenGoalTeam
        #     print oddArrayTemp
        #     for y in range(0,len(oddArrayTemp)/2):
        #         if oddGoalTeam[y]==homeTeam:
        #             oddArrayTemp[2*y].append("W")
        #             oddArrayTemp[2*y+1].append("L")
        #         else:
        #             oddArrayTemp[2*y].append("L")
        #             oddArrayTemp[2*y+1].append("W")

        #     print evenArrayTemp
        #     print evenGoalTeam
        #     print evenarrayFailure
        #     print len(evenArrayTemp)
        #     print arrayTemp
        #     print len(evenArrayTemp)/2
        #     print len(evenGoalTeam)
        #     print evenGoalTeam
            evenArrayTemp=filter(lambda a: a != [], evenArrayTemp)
            oddArrayTemp=filter(lambda a: a != [], oddArrayTemp)
            # evenArrayTemp.remove([])
        #     print evenArrayTemp
    #         print oddGoalTeam
    #         print oddArrayTemp
    #         print evenGoalTeam
    #         print evenArrayTemp
            for y in range(0,len(oddArrayTemp)/2):
                if oddGoalTeam[y]==homeTeam:
                    oddArrayTemp[2*y].insert(0,"W")
                    oddArrayTemp[2*y].insert(0,homeTeam)
                    oddArrayTemp[2*y+1].insert(0,"L")
                    oddArrayTemp[2*y+1].insert(0,awayTeam)
                else:
                    oddArrayTemp[2*y].insert(0,"L")
                    oddArrayTemp[2*y].insert(0,homeTeam)
                    oddArrayTemp[2*y+1].insert(0,"W")
                    oddArrayTemp[2*y+1].insert(0,awayTeam)
    #         print evenArrayTemp
    #         print evenGoalTeam

            for y in range(0,len(evenArrayTemp)/2):
                if evenGoalTeam[y]==homeTeam:
                    evenArrayTemp[2*y].insert(0,"W")
                    evenArrayTemp[2*y].insert(0,homeTeam)
                    evenArrayTemp[2*y+1].insert(0,"L")
                    evenArrayTemp[2*y+1].insert(0,awayTeam)
                else:
                    evenArrayTemp[2*y].insert(0,"L")
                    evenArrayTemp[2*y].insert(0,homeTeam)
                    evenArrayTemp[2*y+1].insert(0,"W")
                    evenArrayTemp[2*y+1].insert(0,awayTeam)

        #     for y in range(0,len(evenArrayTemp)/2):
        #     #     print y
        #         if evenGoalTeam[y]==homeTeam:
        #             evenArrayTemp[2*y].append("W")
        #             evenArrayTemp[2*y+1].append("L")
        #         else:
        #             evenArrayTemp[2*y].append("L")
        #             evenArrayTemp[2*y+1].append("W")

        #     print 
        #     print result2[1]
        #     print result2[2]
        #     print result2[3]
        #     print indicesloc
    #         print EVENGoals
    #         print result2[0]
    #         print result2[1]
    #         print result2[2]
    #         print result2[3]
    #         print totalGoals
    #         print indicesloc
#                 print "result2[0]"
#                 print result2[0]
#                 print result2[1]
#                 print result2[2]
    #         if len(resultEven[x])==3:
            for x in range(0,len(indicesloc)):
            #     print x
    #             if totalGoals%2==0 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
                if totalGoals%2==0:
            #         evenGoalTeam.append((str(result2TeamEven[indicesloc[x]][4].text.strip())))
    #                 evenGoals.append(result2[EVENGoals][0])
    #                 evenFstAssists.append(result2[EVENGoals][1])
    #                 evenSndAssists.append(result2[EVENGoals][2])
                    if len(result2[ODDGoals])==3:
                        oddGoals.append(result2[ODDGoals][0])
                        oddFstAssists.append(result2[ODDGoals][1])
                        oddSndAssists.append(result2[ODDGoals][2])
    #             elif totalGoals%2==1 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
                elif totalGoals%2==1:
            #         oddGoalTeam.append((str(resultTeam2[indicesloc[x]][4].text.strip())))
    #                 oddGoals.append(result2[ODDGoals][0])
    #                 oddFstAssists.append(result2[ODDGoals][1])
    #                 oddSndAssists.append(result2[ODDGoals][2])
#                     print "Yeah"
#                     print EVENGoals
#                     print result2TeamEven[0]
#                     print result2TeamEven[1]
#                     print result2TeamEven[2]
#                     print "resultEven[EVENGoals]"
#                     print resultEven[3]
#                     print resultEven[EVENGoals]
#                     print len(result2TeamEven[EVENGoals])
                    if len(resultEven[EVENGoals])==3:
#                         print "Yeah1"
                        evenGoals.append(resultEven[EVENGoals][0])
                        evenFstAssists.append(resultEven[EVENGoals][1])
                        evenSndAssists.append(resultEven[EVENGoals][2])

            # oddGoals.append(result2[ODDGoals][0])
            # oddFstAssists.append(result2[ODDGoals][1])
            # oddSndAssists.append(result2[ODDGoals][2])

            oddGoalsNumber= [str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddGoals]
            oddFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddFstAssists]
            oddSndNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddSndAssists]
            evenGoalsNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenGoals]
            evenFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenFstAssists]
            evenSndNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenSndAssists ]
        #     print "evenGoalsNumber"
        #     print oddGoals
            # for x in range(0,len(indicesloc)):
            #     print x
            #     if indicesloc[x]%2==0:
            #         evenGoalsNumber.append((str(result2TeamEven[indicesloc[x]][4].text.strip())))
            #     else:
            #         oddGoalsNumber.append((str(resultTeam2[indicesloc[x]][4].text.strip())))

            goalOdd = []
            fstAssistOdd = []
            sndAssistOdd = []

            for y in range(0,len(oddGoalTeam)):
                goalOdd.append([])
                fstAssistOdd.append([])
                sndAssistOdd.append([])
#             print oddGoalsNumber
        #     print oddFstNumber
        #     print oddSndNumber
#             print oddGoalTeam

            for y in range(0,len(oddGoalTeam)):
                goalOdd[y].append(oddGoalsNumber[y])
                fstAssistOdd[y].append(oddFstNumber[y])
                sndAssistOdd[y].append(oddSndNumber[y])

            goalEven = []
            fstAssistEven = []
            sndAssistEven = []

            for y in range(0,len(evenGoalTeam)):
                goalEven.append([])
                fstAssistEven.append([])
                sndAssistEven.append([])

#             print evenGoalsNumber
#             print evenGoals
        #     print evenFstNumber
        #     print evenSndNumber
#             print evenGoalTeam
        #     print evenarrayFailure

            for y in range(0,len(evenGoalTeam)):
                goalEven[y].append(evenGoalsNumber[y])
                fstAssistEven[y].append(evenFstNumber[y])
                sndAssistEven[y].append(evenSndNumber[y])


            for y in range(0,len(goalEven)):
                if evenGoalTeam[y]==homeTeam:
                    goalEven[y].insert(0,str(homeTeam))
                    fstAssistEven[y].insert(0,str(homeTeam))
                    sndAssistEven[y].insert(0,str(homeTeam))
                else:
                    goalEven[y].insert(0,str(awayTeam))
                    fstAssistEven[y].insert(0,str(awayTeam))
                    sndAssistEven[y].insert(0,str(awayTeam))

        #     print "goalEven"
        #     print goalEven
        #     print fstAssistEven
        #     print sndAssistEven
    #         print oddGoalTeam
    #         print goalOdd
            for y in range(0,len(goalOdd)):
                if oddGoalTeam[y]==homeTeam:
                    goalOdd[y].insert(0,str(homeTeam))
                    fstAssistOdd[y].insert(0,str(homeTeam))
                    sndAssistOdd[y].insert(0,str(homeTeam))
                else:
                    goalOdd[y].insert(0,str(awayTeam))
                    fstAssistOdd[y].insert(0,str(awayTeam))
                    sndAssistOdd[y].insert(0,str(awayTeam))

        #     print "goalOdd"
        #     print goalOdd
        #     print fstAssistOdd
        #     print sndAssistOdd



        #     print goalEven+goalOdd
        #     print fstAssistEven+fstAssistOdd
        #     print sndAssistEven+sndAssistOdd
            listEvenManPower=[]
            listOddManPower=[]
            for num in range(0,EVENGoals1):
#                 print "resultCenEven[0]"
#                 print resultCenEven[num][3]
                listEvenManPower.append(resultCenEven[num][3].string)
            for num in range(0,ODDGoals1):
#                 print "resultCen[0]"
#                 print resultCen[num][3]
                listOddManPower.append(resultCen[num][3].string)
#             print "listEvenManPower--"
#             print listEvenManPower
#             print "listOddManPower--"
#             print listOddManPower
#             listEvenManPower=filter(lambda a: a != None, listEvenManPower)
#             listOddManPower=filter(lambda a: a != None, listOddManPower)
            listEvenManPower=filter(lambda a: a != None, listEvenManPower)
            listOddManPower=filter(lambda a: a != None, listOddManPower)
            listOddManPower=filter(lambda a: str(a).isdigit() == False, listOddManPower)
            listEvenManPower=filter(lambda a: str(a).isdigit() == False, listEvenManPower)
#             listOddManPower=filter(lambda a: "-PS" in a == False, listOddManPower)
#             listEvenManPower=filter(lambda a: "-PS" in a == False, listEvenManPower)
            listOddManPower1=[]
            for elem in listOddManPower:
                if "-PS" not in str(elem):
                    listOddManPower1.append(elem)
            listOddManPower=listOddManPower1
            listEvenManPower1=[]
            for elem in listEvenManPower:
                if "-PS" not in str(elem):
                    listEvenManPower1.append(elem)
            listEvenManPower=listEvenManPower1
#             print "listEvenManPower1--"
#             print listEvenManPower
#             print "listOddManPower1--"
#             print listOddManPower
            goalEvenPP=[]
            fstAssistEvenPP=[]
            sndAssistEvenPP=[]
            evenArrayTempPP=[]
            for i in range(0,len(listEvenManPower)):
                if listEvenManPower[i]=="PP":
                    goalEvenPP.append(goalEven[i])
                    fstAssistEvenPP.append(fstAssistEven[i])
                    sndAssistEvenPP.append(sndAssistEven[i])
                    evenArrayTempPP.append(evenArrayTemp[2*i])
                    evenArrayTempPP.append(evenArrayTemp[2*i+1])
            goalOddPP=[]
            fstAssistOddPP=[]
            sndAssistOddPP=[]
            oddArrayTempPP=[]
            print goalOdd
            print listOddManPower
            for i in range(0,len(listOddManPower)):
                if listOddManPower[i]=="PP":
                    goalOddPP.append(goalOdd[i])   
                    fstAssistOddPP.append(fstAssistOdd[i])
                    sndAssistOddPP.append(sndAssistOdd[i])
                    oddArrayTempPP.append(oddArrayTemp[2*i])
                    oddArrayTempPP.append(oddArrayTemp[2*i+1])
#             print "goalEvenPP-"
#             print goalEvenPP
            print "temp----"
            print temp
            
            GoalPP=goalEvenPP+goalOddPP
            FstAssistPP=fstAssistEvenPP+fstAssistOddPP
            SndAssistPP=sndAssistEvenPP+sndAssistOddPP
            ArrayTempPP=evenArrayTempPP+oddArrayTempPP
            
            Scorers.append(GoalPP)
            Scorers3.append(GoalPP)
            FstAssistsPlayers.append(FstAssistPP)
            SndAssistsPlayers.append(SndAssistPP)
            plusMinusPlayers.append(ArrayTempPP)
    elif numGames in [20036,20078,20232,20235,20290,20332,20441,20461,20544,20547,20755,20812,20846,20903,21025,21114,21144,21151,21164,21176]: 
        print numGames
#         soup = BeautifulSoup(get_poll_xml1(numGames))
#         # soup

#         result1= soup.find_all("tr", attrs={"class":"oddColor"} )

#         result2=[elem.find_all("td", attrs={"align":"left"} ) for elem in result1]

#         resultEven= soup.find_all("tr", attrs={"class":"evenColor"} )

#         resultEven=[elem.find_all("td", attrs={"align":"left"} ) for elem in resultEven]

#         result3= soup.find_all("td", attrs={"align":"center", "style":"font-size: 40px;font-weight:bold"} )

#         totalGoals=sum([int(elem.text.strip()) for elem in result3])
#         resultCen=[elem.find_all("td", attrs={"align":"center"} ) for elem in result1]
#         print numGames
        soup = BeautifulSoup(get_poll_xml1(numGames))
        # soup

        result1= soup.find_all("tr", attrs={"class":"oddColor"} )
#         resultCenter= soup.find_all("td", attrs={"align":"center","class":"oddColor"} )
#         resultCen= result1.find_all("td", attrs={"align":"center"} )
        resultCen=[elem.find_all("td", attrs={"align":"center"} ) for elem in result1]
#         print resultCen
        
        result2=[elem.find_all("td", attrs={"align":"left"} ) for elem in result1]

        resultEven1= soup.find_all("tr", attrs={"class":"evenColor"} )

        resultEven=[elem.find_all("td", attrs={"align":"left"} ) for elem in resultEven1]

        result3= soup.find_all("td", attrs={"align":"center", "style":"font-size: 40px;font-weight:bold"} )

        totalGoals=sum([int(elem.text.strip()) for elem in result3])
        resultCenEven=[elem.find_all("td", attrs={"align":"center"} ) for elem in resultEven1]
        if totalGoals%2==0:
            ODDGoals=totalGoals/2
            EVENGoals=totalGoals/2
        else:
            ODDGoals=totalGoals/2+1
            EVENGoals=totalGoals/2

        oddGoals=[]
        oddFstAssists=[]
        oddSndAssists=[]
        oddarrayFailure=[]
        for x in range(0,ODDGoals):
            if len(result2[x])==3:
                oddGoals.append(result2[x][0])
                oddFstAssists.append(result2[x][1])
                oddSndAssists.append(result2[x][2])
            else:
                oddarrayFailure.append(x)

        evenGoals=[]
        evenFstAssists=[]
        evenSndAssists=[]    
        evenarrayFailure=[]
        for x in range(0,EVENGoals):
        #     print "Test"
        #     print 
            if len(resultEven[x])==3:
                evenGoals.append(resultEven[x][0])
        #         print resultEven[x][1]
                evenFstAssists.append(resultEven[x][1])
                evenSndAssists.append(resultEven[x][2])
            else:
                evenarrayFailure.append(x)
        #     else:
        #         evenGoals.append(resultEven[x][0])
        # #         print resultEven[x][1]
        #         evenFstAssists.append(resultEven[x][1])
        #         evenSndAssists.append()



        resultplayers=soup.find_all("font", attrs={"style":"cursor:hand;"} )

        tds=soup.find_all("td" )[1:]
        tds=[str(elem) for elem in tds]

        count=[]
        tds2=[]
    #     print tds

        for x in range(0,len(tds)):
            if "font style=" in tds[x]:
                count.append(tds[x].count ("font style="))
                tds2.append(tds[x])
    #     print tds2[2]
    #     print tds2
        if tds2!=[]:
            indices = [i for i, x in enumerate(tds2[2]) if x == ","]
        #     print tds2[2]
            temp=[]
            for x in range(0,len(indices)):
                if x==0:
                    temp.append(tds2[2][:indices[0]])
                    temp.append(tds2[2][indices[x]:indices[x+1]])
                elif x!=len(indices)-1:
                    temp.append(tds2[2][indices[x]:indices[x+1]])
                elif x==len(indices)-1:
                    temp.append(tds2[2][indices[x]:len(tds2[2])])
    #         print temp
            if temp!=[]:
                temp[0]=temp[0].replace("<td>", "")
        #     print temp

        #     print temp
            for x in range(0,len(temp)):
                temp[x]=temp[x][(temp[x].index(">")+1):len(temp[x])]
                temp[x]=temp[x][0:(temp[x].index("<"))] 
            arrayTemp=[]

            indices = []

            arrayTemp=[]
            for y in range(1,len(tds2)):
                temp=[]
                indices = [i for i, x in enumerate(tds2[y]) if x == ","]
            #     print tds2[y]
            #     print indices
                for x in range(0,len(indices)):
                    if x==0:
                        temp.append(tds2[y][:indices[0]])
                        temp.append(tds2[y][indices[x]:indices[x+1]])
                    elif x!=len(indices)-1:
                        temp.append(tds2[y][indices[x]:indices[x+1]])
                    elif x==len(indices)-1:
                        temp.append(tds2[y][indices[x]:len(tds2[y])])
        #         print y
        #         print temp
                if temp!=[]:
                    temp[0]=temp[0].replace("<td>", "")
            #     print y
            #     print temp
                for x in range(0,len(temp)):
                    temp[x]=temp[x][(temp[x].index(">")+1):len(temp[x])]
                    temp[x]=temp[x][0:(temp[x].index("<"))]
                arrayTemp.append(temp)



            resultTeam=[elem.find_all("td", attrs={"align":"left","class":"heading + bborder"} ) for elem in soup]
            homeTeam=resultTeam[0][3].text.strip()[:resultTeam[0][3].text.strip().index(" ")]
            awayTeam=resultTeam[0][4].text.strip()[:resultTeam[0][4].text.strip().index(" ")]
            resultTeam1= soup.find_all("tr", attrs={"class":"oddColor"} )
            resultTeam2=[elem.find_all("td", attrs={"align":"center"} ) for elem in resultTeam1]

            result1TeamEven= soup.find_all("tr", attrs={"class":"evenColor"} )
            result2TeamEven=[elem.find_all("td", attrs={"align":"center"} ) for elem in result1TeamEven]
            oddGoalTeam=[]
            evenGoalTeam=[]
        #     print "EVENGoals"
        #     print ODDGoals
        #     print EVENGoals
        #     print arrayTemp
        #     print "evenarrayFailure"
        #     print evenarrayFailure
        #     # oddFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddFstAssists]
        #     # print indicesloc

        #     print arrayTemp
    #         print 
            for x in range(0,ODDGoals):
                if x  not in oddarrayFailure:
                    oddGoalTeam.append(str(resultTeam2[x][4].text.strip()))
#             print evenarrayFailure
            for x in range(0,EVENGoals):
        #         print x
                if x not in evenarrayFailure:
        #             print x
                    evenGoalTeam.append(str(result2TeamEven[x][4].text.strip()))

            indicesloc = [i for i, x in enumerate(arrayTemp) if x == []]
            # print arrayTemp.index([])
        #     print indicesloc 
            # indicesloc=[]
            indicesloc= [ elem/2  for elem in indicesloc if elem%2==0]
        #     print "result2TeamEven[indicesloc[0]]"
        #     print indicesloc
        #     print result2TeamEven[0]
        #     print result2TeamEven[1]
        #     print result2TeamEven[2]
        #     print result2TeamEven[3]
        #     print result2TeamEven[4]
        #     print result2TeamEven[5]
        #     print "result2TeamEven[indicesloc[0]/2]"
            # print result2TeamEven[indicesloc[0]/2]

            # print result2TeamEven
    #         print indicesloc
    #         print totalGoals
            # print result2TeamEven[indicesloc[0]]
    #         print evenGoalTeam
    #         for x in range(0,len(indicesloc)):
    # #             print x
    # #             if totalGoals%2==0 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
    #             if totalGoals%2==0:
    #                 print "Even!"
    #                 oddGoalTeam.append((str(resultTeam2[indicesloc[x]/2][4].text.strip())))
    # #                 evenGoalTeam.append((str(result2TeamEven[indicesloc[x]/2][4].text.strip())))
    # #             elif totalGoals%2==1 and (indicesloc[x]!=totalGoals  and indicesloc[x]!=totalGoals-1):
    #             elif totalGoals%2==1:
    #                 print "Odd!"
    # #                 oddGoalTeam.append((str(resultTeam2[indicesloc[x]/2][4].text.strip())))
    #                 evenGoalTeam.append((str(result2TeamEven[indicesloc[x]/2][4].text.strip())))
    # #         print indicesloc
    # #         print totalGoals
    # #         print evenGoalTeam
    #         [1]
    # 3
    # []
            oddArrayTemp=[]
            evenArrayTemp=[]
            for y in range(0,len(arrayTemp)):
                if y%4==1:
                    oddArrayTemp.append(arrayTemp[y]) 
                elif y%4==0:
                    oddArrayTemp.append(arrayTemp[y]) 
                elif y%4==2:
                    evenArrayTemp.append(arrayTemp[y]) 
                elif y%4==3:
                    evenArrayTemp.append(arrayTemp[y])
        #     print "oddArrayTemp"   
            # print arrayTemp.index([])

        #     print oddGoalTeam
        #     print evenGoalTeam
        #     print oddArrayTemp
        #     for y in range(0,len(oddArrayTemp)/2):
        #         if oddGoalTeam[y]==homeTeam:
        #             oddArrayTemp[2*y].append("W")
        #             oddArrayTemp[2*y+1].append("L")
        #         else:
        #             oddArrayTemp[2*y].append("L")
        #             oddArrayTemp[2*y+1].append("W")

        #     print evenArrayTemp
        #     print evenGoalTeam
        #     print evenarrayFailure
        #     print len(evenArrayTemp)
        #     print arrayTemp
        #     print len(evenArrayTemp)/2
        #     print len(evenGoalTeam)
        #     print evenGoalTeam
            evenArrayTemp=filter(lambda a: a != [], evenArrayTemp)
            oddArrayTemp=filter(lambda a: a != [], oddArrayTemp)
            # evenArrayTemp.remove([])
        #     print evenArrayTemp
#             print "oddArrayTemp----"
            if numGames==20544:
                oddGoalTeam.append("VAN")
#             print oddGoalTeam
#             print oddArrayTemp
            for y in range(0,len(oddArrayTemp)/2):
                if oddGoalTeam[y]==homeTeam:
                    oddArrayTemp[2*y].insert(0,"W")
                    oddArrayTemp[2*y].insert(0,homeTeam)
                    oddArrayTemp[2*y+1].insert(0,"L")
                    oddArrayTemp[2*y+1].insert(0,awayTeam)
                else:
                    oddArrayTemp[2*y].insert(0,"L")
                    oddArrayTemp[2*y].insert(0,homeTeam)
                    oddArrayTemp[2*y+1].insert(0,"W")
                    oddArrayTemp[2*y+1].insert(0,awayTeam)
#             print evenArrayTemp
#             print evenGoalTeam
# #             print evenArrayTemp
#             print oddGoalTeam
            if numGames==20332:
                evenGoalTeam=["NSH"]
#             print "evenArrayTemp----"
            for y in range(0,len(evenArrayTemp)/2):
                if evenGoalTeam[y]==homeTeam:
                    evenArrayTemp[2*y].insert(0,"W")
                    evenArrayTemp[2*y].insert(0,homeTeam)
                    evenArrayTemp[2*y+1].insert(0,"L")
                    evenArrayTemp[2*y+1].insert(0,awayTeam)
                else:
                    evenArrayTemp[2*y].insert(0,"L")
                    evenArrayTemp[2*y].insert(0,homeTeam)
                    evenArrayTemp[2*y+1].insert(0,"W")
                    evenArrayTemp[2*y+1].insert(0,awayTeam)

        #     for y in range(0,len(evenArrayTemp)/2):
        #     #     print y
        #         if evenGoalTeam[y]==homeTeam:
        #             evenArrayTemp[2*y].append("W")
        #             evenArrayTemp[2*y+1].append("L")
        #         else:
        #             evenArrayTemp[2*y].append("L")
        #             evenArrayTemp[2*y+1].append("W")

        #     print 
        #     print result2[1]
        #     print result2[2]
        #     print result2[3]
        #     print indicesloc
    #         print EVENGoals
    #         print result2[0]
    #         print result2[1]
    #         print result2[2]
    #         print result2[3]
    #         print totalGoals
    #         print indicesloc
#             print result2[EVENGoals]
    #         if len(resultEven[x])==3:
            for x in range(0,len(indicesloc)):
            #     print x
    #             if totalGoals%2==0 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
                if totalGoals%2==0:
            #         evenGoalTeam.append((str(result2TeamEven[indicesloc[x]][4].text.strip())))
    #                 evenGoals.append(result2[EVENGoals][0])
    #                 evenFstAssists.append(result2[EVENGoals][1])
    #                 evenSndAssists.append(result2[EVENGoals][2])
                    if len(result2[ODDGoals])==3:
                        oddGoals.append(result2[ODDGoals][0])
                        oddFstAssists.append(result2[ODDGoals][1])
                        oddSndAssists.append(result2[ODDGoals][2])
    #             elif totalGoals%2==1 and (indicesloc[x]!=totalGoals and indicesloc[x]!=totalGoals-1):
                elif totalGoals%2==1:
            #         oddGoalTeam.append((str(resultTeam2[indicesloc[x]][4].text.strip())))
    #                 oddGoals.append(result2[ODDGoals][0])
    #                 oddFstAssists.append(result2[ODDGoals][1])
    #                 oddSndAssists.append(result2[ODDGoals][2])
#                     print "Yeah"
#                     print EVENGoals
#                     print result2TeamEven[0]
#                     print result2TeamEven[1]
#                     print result2TeamEven[2]
#                     print " result2TeamEven[3]"
#                     print result2TeamEven[3]
#                     print result2[ODDGoals]
#                     print len(result2TeamEven[EVENGoals])
                    if len(resultEven[EVENGoals])==3:
#                         print "Yeah1"
                        evenGoals.append(resultEven[EVENGoals][0])
                        evenFstAssists.append(resultEven[EVENGoals][1])
                        evenSndAssists.append(resultEven[EVENGoals][2])
            if totalGoals1%2==0:
                strtemp=str(resultCen[EVENGoals1][1])[(str(resultCen[EVENGoals1][1]).index(">"))+1:]
                strtemp=strtemp[0:(strtemp.index("<"))] 
                manPower=str(resultCen[EVENGoals1][3])[(str(resultCen[EVENGoals1][3]).index(">"))+1:]
                manPower=manPower[0:(manPower.index("<"))] 
            else:
                strtemp=str(resultCen[ODDGoals1][1])[(str(resultCen[ODDGoals1][1]).index(">"))+1:]
                strtemp=strtemp[0:(strtemp.index("<"))]
                manPower=str(resultCen[ODDGoals1][3])[(str(resultCen[ODDGoals1][3]).index(">"))+1:]
                manPower=manPower[0:(manPower.index("<"))] 
            # oddGoals.append(result2[ODDGoals][0])
            # oddFstAssists.append(result2[ODDGoals][1])
            # oddSndAssists.append(result2[ODDGoals][2])

            oddGoalsNumber= [str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddGoals]
            oddFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddFstAssists]
            oddSndNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in oddSndAssists]
            evenGoalsNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenGoals]
            evenFstNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenFstAssists]
            evenSndNumber=[str(elem.text.strip())[0:str(elem.text.strip()).index(" ")] if " " in str(elem.text.strip()) else str(elem.text.strip()) for elem in evenSndAssists ]
        #     print "evenGoalsNumber"
        #     print oddGoals
            # for x in range(0,len(indicesloc)):
            #     print x
            #     if indicesloc[x]%2==0:
            #         evenGoalsNumber.append((str(result2TeamEven[indicesloc[x]][4].text.strip())))
            #     else:
            #         oddGoalsNumber.append((str(resultTeam2[indicesloc[x]][4].text.strip())))

            goalOdd = []
            fstAssistOdd = []
            sndAssistOdd = []

            for y in range(0,len(oddGoalTeam)):
                goalOdd.append([])
                fstAssistOdd.append([])
                sndAssistOdd.append([])
#             print oddGoalsNumber
        #     print oddFstNumber
        #     print oddSndNumber
#             print oddGoalTeam

            for y in range(0,len(oddGoalTeam)):
                goalOdd[y].append(oddGoalsNumber[y])
                fstAssistOdd[y].append(oddFstNumber[y])
                sndAssistOdd[y].append(oddSndNumber[y])

            goalEven = []
            fstAssistEven = []
            sndAssistEven = []

            for y in range(0,len(evenGoalTeam)):
                goalEven.append([])
                fstAssistEven.append([])
                sndAssistEven.append([])

#             print evenGoalsNumber
#             print evenGoals
        #     print evenFstNumber
        #     print evenSndNumber
#             print evenGoalTeam
        #     print evenarrayFailure

            for y in range(0,len(evenGoalTeam)):
                goalEven[y].append(evenGoalsNumber[y])
                fstAssistEven[y].append(evenFstNumber[y])
                sndAssistEven[y].append(evenSndNumber[y])


            for y in range(0,len(goalEven)):
                if evenGoalTeam[y]==homeTeam:
                    goalEven[y].insert(0,str(homeTeam))
                    fstAssistEven[y].insert(0,str(homeTeam))
                    sndAssistEven[y].insert(0,str(homeTeam))
                else:
                    goalEven[y].insert(0,str(awayTeam))
                    fstAssistEven[y].insert(0,str(awayTeam))
                    sndAssistEven[y].insert(0,str(awayTeam))

        #     print "goalEven"
        #     print goalEven
        #     print fstAssistEven
        #     print sndAssistEven
    #         print oddGoalTeam
    #         print goalOdd
            for y in range(0,len(goalOdd)):
                if oddGoalTeam[y]==homeTeam:
                    goalOdd[y].insert(0,str(homeTeam))
                    fstAssistOdd[y].insert(0,str(homeTeam))
                    sndAssistOdd[y].insert(0,str(homeTeam))
                else:
                    goalOdd[y].insert(0,str(awayTeam))
                    fstAssistOdd[y].insert(0,str(awayTeam))
                    sndAssistOdd[y].insert(0,str(awayTeam))

#             print "evenOdd"
#             print goalEven
        #     print fstAssistOdd
        #     print sndAssistOddresultEven



        #     print goalEven+goalOdd
        #     print fstAssistEven+fstAssistOdd
        #     print sndAssistEven+sndAssistOdd
#             print "strtempYeah"
#             print strtemp
#             print resultCen[ODDGoals1][1]
#             if numGames==20232:
#                 print "20232-"
#                 print goalOdd
#                 print fstAssistOdd
#                 print sndAssistOdd
#                 del goalOdd[-1]
#                 del goalOdd[-1]
            listEvenManPower=[]
            listOddManPower=[]
#             print "resultCenEven"
#             print resultCenEven
#             print "resultCen"
#             print resultCen
            for num in range(0,EVENGoals1):
#                 print "resultCenEven[0]"
#                 print resultCenEven[num][3]
                listEvenManPower.append(resultCenEven[num][3].string)
            for num in range(0,ODDGoals1):
#                 print "resultCen[0]"
#                 print resultCen[num][3]
                listOddManPower.append(resultCen[num][3].string)
#             print "listEvenManPower--"
#             print listEvenManPower
#             print "listOddManPower--"
#             print listOddManPower
#             listEvenManPower=filter(lambda a: a != None, listEvenManPower)
#             listOddManPower=filter(lambda a: a != None, listOddManPower)
            listEvenManPower=filter(lambda a: a != None, listEvenManPower)
            listOddManPower=filter(lambda a: a != None, listOddManPower)
            listOddManPower=filter(lambda a: str(a).isdigit() == False, listOddManPower)
            listEvenManPower=filter(lambda a: str(a).isdigit() == False, listEvenManPower)
#             listOddManPower=filter(lambda a: "-PS" in a == False, listOddManPower)
#             listEvenManPower=filter(lambda a: "-PS" in a == False, listEvenManPower)
            listOddManPower1=[]
            for elem in listOddManPower:
                if "-PS" not in str(elem):
                    listOddManPower1.append(elem)
            listOddManPower=listOddManPower1
            listEvenManPower1=[]
            for elem in listEvenManPower:
#                 print "str(elem)"
#                 print str(elem)
                if "-PS" not in str(elem):
#                     print "str(elem)1"
#                     print str(elem)
                    listEvenManPower1.append(elem)
            listEvenManPower=listEvenManPower1
#             print "listEvenManPower1--"
#             print listEvenManPower
#             print "listOddManPower1--"
#             print listOddManPower
            goalEvenPP=[]
            fstAssistEvenPP=[]
            sndAssistEvenPP=[]
            evenArrayTempPP=[]
#             print "goalEven"
#             print goalEven
#             print "listEvenManPower"
#             print listEvenManPower
            for i in range(0,len(listEvenManPower)):
                if listEvenManPower[i]=="PP":
                    goalEvenPP.append(goalEven[i])
                    fstAssistEvenPP.append(fstAssistEven[i])
                    sndAssistEvenPP.append(sndAssistEven[i])
                    evenArrayTempPP.append(evenArrayTemp[2*i])
                    evenArrayTempPP.append(evenArrayTemp[2*i+1])
            goalOddPP=[]
            fstAssistOddPP=[]
            sndAssistOddPP=[]
            oddArrayTempPP=[]
            
#             print "goalOdd"
#             print goalOdd
#             print "listOddManPower"
#             print listOddManPower
            for i in range(0,len(listOddManPower)):
                if listOddManPower[i]=="PP":
                    goalOddPP.append(goalOdd[i])   
                    fstAssistOddPP.append(fstAssistOdd[i])
                    sndAssistOddPP.append(sndAssistOdd[i])
                    oddArrayTempPP.append(oddArrayTemp[2*i])
                    oddArrayTempPP.append(oddArrayTemp[2*i+1])
#             print "goalEvenPP-"
#             print goalEvenPP
#             print "temp----"
#             print temp
            
            GoalPP=goalEvenPP+goalOddPP
            FstAssistPP=fstAssistEvenPP+fstAssistOddPP
            SndAssistPP=sndAssistEvenPP+sndAssistOddPP
            ArrayTempPP=evenArrayTempPP+oddArrayTempPP
            Goal=goalEven+goalOdd
            FstAssist=fstAssistEven+fstAssistOdd
            SndAssist=sndAssistEven+sndAssistOdd
            ArrayTemp=evenArrayTemp+oddArrayTemp
            if strtemp=="SO":
#                 del ArrayTemp[-1]
                del FstAssist[-1]
                del SndAssist[-1]
                del Goal[-1]
            Scorers.append(GoalPP)
            Scorers3.append(GoalPP)
            FstAssistsPlayers.append(FstAssistPP)
            SndAssistsPlayers.append(SndAssistPP)
            plusMinusPlayers.append(ArrayTempPP)
print "Scorers1--"
import copy
Scorers1=copy.copy(Scorers)
print Scorers1
print Scorers
# print Scorers
newScorers=[]
# Scorers1=Scorers[:]
# print Scorers1
for x in range(0,len(Scorers1)):
    newScorers=newScorers+Scorers1[x]
# print newScorers
# print "Scorers2"
# print Scorers1
# print Scorers
newFstAssistsPlayers=[]
for x in range(0,len(FstAssistsPlayers)):
    newFstAssistsPlayers=newFstAssistsPlayers+FstAssistsPlayers[x]
# print newFstAssistsPlayers
newSndAssistsPlayers=[]
for x in range(0,len(SndAssistsPlayers)):
    newSndAssistsPlayers=newSndAssistsPlayers+SndAssistsPlayers[x]
# print newSndAssistsPlayers
# print len(newScorers)
# print len(newFstAssistsPlayers)
# print len(newSndAssistsPlayers)
# print FstAssistsPlayers
# print SndAssistsPlayers
# print len(Scorers)
# print len(FstAssistsPlayers)
# print len(SndAssistsPlayers)
# print plusMinusPlayers
# print len(plusMinusPlayers)
# print np.unique(newScorers)
# print list(set(zip(*newScorers)[1]))
# uniqueNewScorers=[]
# for x in range(0,len(newScorers)):
#     if newScorers[x] not in uniqueNewScorers:
#         uniqueNewScorers.append(newScorers[x])
# print uniqueNewScorers
# print len(uniqueNewScorers)
# sortedUniqueNewScorers= sorted(uniqueNewScorers)

# for x in range(0,len(sortedUniqueNewScorers)):
#     counter=0
#     for y in range(0,len(newScorers)):
#         if sortedUniqueNewScorers[x]==newScorers[y]:
#             counter=counter+1
#     sortedUniqueNewScorers[x].append(counter)

# print sortedUniqueNewScorers
# print "Important24"
# print FstAssistsPlayers
unassistedGoals=[]
FstassistedGoals=[]
SndassistedGoals=[]
onlyFstassist=[]
SndassistedFstassist=[]


# print "Important25"
# print FstAssistsPlayers
for x in range(0,len(newScorers)):
    if str(newFstAssistsPlayers[x][1])=='unassisted':
        unassistedGoals.append(newScorers[x])
    else:
        if str(newSndAssistsPlayers[x][1])=='':
            FstassistedGoals.append(newScorers[x])
            onlyFstassist.append(newFstAssistsPlayers[x])
            
        else:
            SndassistedGoals.append(newScorers[x])
            SndassistedFstassist.append(newFstAssistsPlayers[x])
# print "Scorers3"
# print Scorers1
# print Scorers

#             newSndAssistsPlayers
        
# print "Important4"
# print newScorers
# print newFstAssistsPlayers
# print newSndAssistsPlayers
# print plusMinusPlayers
# print "Important5"
# print FstAssistsPlayers
# print FstAssistsPlayers1
# print newFstAssistsPlayers
# print newFstAssistsPlayers[0]
# print unassistedGoals
# print FstassistedGoals
# print SndassistedGoals
# print onlyFstassist
# print SndassistedFstassist
# print newSndAssistsPlayers

uniqueNewScorers=[]
for x in range(0,len(newScorers)):
    if newScorers[x] not in uniqueNewScorers:
        uniqueNewScorers.append(newScorers[x])
# print uniqueNewScorers
# print len(uniqueNewScorers)
# print "Scorers4"
# print Scorers1
# print Scorers
sortedUniqueNewScorers= sorted(uniqueNewScorers)
# print "Scorers5"
# print Scorers1
# print Scorers


uniqueNewFstAssistsPlayers=[]
# print FstAssistsPlayers
# FstAssistsPlayers1=FstAssistsPlayers
for x in range(0,len(newFstAssistsPlayers)):
    if newFstAssistsPlayers[x] not in uniqueNewFstAssistsPlayers:
        uniqueNewFstAssistsPlayers.append(newFstAssistsPlayers[x])

sortedUniqueNewFstAssistsPlayers= sorted(uniqueNewFstAssistsPlayers)
print "Important23"
print onlyFstassist
print sortedUniqueNewFstAssistsPlayers



uniqueNewSndAssistsPlayers=[]
for x in range(0,len(newSndAssistsPlayers)):
    if newSndAssistsPlayers[x] not in uniqueNewSndAssistsPlayers:
        uniqueNewSndAssistsPlayers.append(newSndAssistsPlayers[x])

sortedUniqueNewSndAssistsPlayers= sorted(uniqueNewSndAssistsPlayers)

uniquePointsPlayers=sortedUniqueNewFstAssistsPlayers[:]
for x in range(0,len(sortedUniqueNewSndAssistsPlayers)):
    if sortedUniqueNewSndAssistsPlayers[x] not in sortedUniqueNewFstAssistsPlayers:
        uniquePointsPlayers.append(sortedUniqueNewSndAssistsPlayers[x])
        
for x in range(0,len(sortedUniqueNewScorers)):
    if sortedUniqueNewScorers[x] not in uniquePointsPlayers:
        uniquePointsPlayers.append(sortedUniqueNewScorers[x])

        
# print uniquePointsPlayers2
# print "Scorers6"
# print Scorers1
# print Scorers
# print "Important33"
# print sortedUniqueNewFstAssistsPlayers
# print sortedUniqueNewSndAssistsPlayers
# print sortedUniqueNewScorers
# print uniquePointsPlayers

# print unassistedGoals
# print FstassistedGoals
# print SndassistedGoals
# print onlyFstassist
# print SndassistedFstassist
# print newSndAssistsPlayers
uniquePointsPlayers1=uniquePointsPlayers[:]
Scores2=Scorers[:]
# print unassistedGoals[0]
# print uniquePointsPlayers[5]
# print unassistedGoals[0]==uniquePointsPlayers[5]

newplusMinusPlayers=[]
for x in range(0,len(plusMinusPlayers)):
    newplusMinusPlayers=newplusMinusPlayers+plusMinusPlayers[x]
    
    
print newplusMinusPlayers
uniquePointsPlayers2=[]
# for x in range(0,len(newplusMinusPlayers[0])):
#     uniquePointsPlayers2.append([])
for x in range(0,len(newplusMinusPlayers)):
    for x in range(0,len(newplusMinusPlayers[x])):
        uniquePointsPlayers2.append([])
# print "uniquePointsPlayers2"
# print uniquePointsPlayers2
# print len(uniquePointsPlayers2)
# print newplusMinusPlayers
# print newplusMinusPlayers[0]
# for x in range(0,len(newplusMinusPlayers)):
#     for y in range(2,len(newplusMinusPlayers[x])):
#         uniquePointsPlayers2[x].append("")
# print uniquePointsPlayers2
# uniquePointsPlayers2=[]
# print 
for x in range(0,len(newplusMinusPlayers)):
    uniquePointsPlayers2[x].append(newplusMinusPlayers[x][0])
    for y in range(2,len(newplusMinusPlayers[x])):
        uniquePointsPlayers2[x].append(newplusMinusPlayers[x][y])
# print uniquePointsPlayers2
arrayUnique2=[]
for x in range(0,len(uniquePointsPlayers2)):
    
    for y in range(2,len(uniquePointsPlayers2[x])):
        unique2=[]
        unique2.append(str(uniquePointsPlayers2[x][0]))
        unique2.append(uniquePointsPlayers2[x][y])
        arrayUnique2.append(unique2)
# print "arrayUnique2"
# print arrayUnique2
# print len(arrayUnique2)
arrayUnique2Store=[]
for x in range(0,len(arrayUnique2)):
    if arrayUnique2[x] not in arrayUnique2Store:
        arrayUnique2Store.append(arrayUnique2[x])

# print arrayUnique2Store
# print len(arrayUnique2Store)
uniquePointsPlayers1=arrayUnique2Store[:]
uniquePointsPlayers=arrayUnique2Store[:]
#         uniquePointsPlayers2[x].append(plusMinusPlayers[y])
# newplusMinusPlayersWin=newplusMinusPlayers[newplusMinusPlayers[1]=='W']
newplusMinusPlayersWin = [row for row in newplusMinusPlayers if 'W' in row[1]]
import copy
temp=copy.deepcopy(newplusMinusPlayersWin)

# print "Scorers"
# print Scorers1
# # print Scorers2
# print newScorers

# print temp
# print FstAssistsPlayers
# print newSndAssistsPlayers
# print newScorers[0][1]
# print "temp[0]"
# print temp[0]
# print "temp[1]"
# print temp[1]
# print "temp[2]"
# print temp[2]
# print "temp[3]"
# print temp[3]
# print "temp[4]"
# print temp[4]
# print "newScorers[0][1]"
# print newScorers[0]
# print "newScorers[1][1]"
# print newScorers[1]
# print "newScorers[2][1]"
# print newScorers[2]
# print "newScorers[3][1]"
# print newScorers[3]
# print "newScorers[4][1]"
# print newScorers[4]
# print temp
for x in range(0,len(newplusMinusPlayersWin)):
    print x
#     print newScorers[x]
#     print temp[x]
    temp[x].remove(newScorers[x][1])
    if newFstAssistsPlayers[x][1]!='unassisted' and newFstAssistsPlayers[x][1] in temp[x]:
        temp[x].remove(newFstAssistsPlayers[x][1])
        if newSndAssistsPlayers[x][1]!='' and newSndAssistsPlayers[x][1] in temp[x]:
            temp[x].remove(newSndAssistsPlayers[x][1])
            temp[x].append("secondAssisted")
        else:
            temp[x].append("firstAssisted")
    else:
         temp[x].append("unassisted")
            
# print "tem"
firstTemp = [row for row in temp if 'firstAssisted' in row[-1]]
secondTemp = [row for row in temp if 'secondAssisted' in row[-1]]
unassistedTemp = [row for row in temp if 'unassisted' in row[-1]]
# print temp
# print firstTemp
# print secondTemp
# print unassistedTemp
# print newplusMinusPlayersWin
# print "uniquePointsPlayers"
# print uniquePointsPlayers
# print uniquePointsPlayers1
# print FstassistedGoals
arrayfirstTemp=[]
for x in range(0,len(firstTemp)):
    
    for y in range(2,len(firstTemp[x])):
        unique3=[]
        unique3.append(str(firstTemp[x][0]))
        unique3.append(firstTemp[x][y])
        arrayfirstTemp.append(unique3)
print "arrayfirstTemp"
print arrayfirstTemp

arraysecondTemp=[]
for x in range(0,len(secondTemp)):
    
    for y in range(2,len(secondTemp[x])):
        unique3=[]
        unique3.append(str(secondTemp[x][0]))
        unique3.append(secondTemp[x][y])
        arraysecondTemp.append(unique3)
# print "arraysecondTemp"
# print arraysecondTemp

arrayunassistedTemp=[]
for x in range(0,len(unassistedTemp)):
    
    for y in range(2,len(unassistedTemp[x])):
        unique3=[]
        unique3.append(str(unassistedTemp[x][0]))
        unique3.append(unassistedTemp[x][y])
        arrayunassistedTemp.append(unique3)
# print "arrayunassistedTemp"
# print arrayunassistedTemp


for x in range(0,len(uniquePointsPlayers)):
    counter=0
    for y in range(0,len(unassistedGoals)):
        if uniquePointsPlayers[x]==unassistedGoals[y]:
            counter=counter+1
    uniquePointsPlayers1[x].append(counter)
    counter=0
    for y in range(0,len(FstassistedGoals)):
        if uniquePointsPlayers[x][:2]==FstassistedGoals[y]:
            counter=counter+1
            print "counter"
            print counter
    uniquePointsPlayers1[x].append(counter) 
    counter=0
    for y in range(0,len(SndassistedGoals)):
        if uniquePointsPlayers[x][:2]==SndassistedGoals[y]:
            counter=counter+1
    uniquePointsPlayers1[x].append(counter) 
    counter=0
    for y in range(0,len(onlyFstassist)):
        if uniquePointsPlayers[x][:2]==onlyFstassist[y]:
            counter=counter+1
    uniquePointsPlayers1[x].append(counter) 
    counter=0
    for y in range(0,len(SndassistedFstassist)):
        if uniquePointsPlayers[x][:2]==SndassistedFstassist[y]:
            counter=counter+1
    uniquePointsPlayers1[x].append(counter) 
    counter=0
    for y in range(0,len(newSndAssistsPlayers)):
        if uniquePointsPlayers[x][:2]==newSndAssistsPlayers[y]:
            counter=counter+1
    uniquePointsPlayers1[x].append(counter) 
    counter=0
    for y in range(0,len(arrayunassistedTemp)-1):
        if uniquePointsPlayers[x][:2]==arrayunassistedTemp[y]:
            counter=counter+1
    uniquePointsPlayers1[x].append(counter)
    counter=0
    for y in range(0,len(arrayfirstTemp)-1):
        if uniquePointsPlayers[x][:2]==arrayfirstTemp[y]:
            counter=counter+1
    uniquePointsPlayers1[x].append(counter)
    counter=0
    for y in range(0,len(arraysecondTemp)-1):
        if uniquePointsPlayers[x][:2]==arraysecondTemp[y]:
            counter=counter+1
    uniquePointsPlayers1[x].append(counter)
    
#     counter=0
#     for y in range(0,len()):
#         if uniquePointsPlayers[x]==newSndAssistsPlayers[y]:
#             counter=counter+1
#     uniquePointsPlayers1[x].append(counter) 
    
    
print "yeah"
# print unassistedGoals
# print uniquePointsPlayers1

# print sortedUniqueNewSndAssistsPlayers
# print "important25"
# print onlyFstassist
# print SndassistedFstassist
# print newSndAssistsPlayers
# print plusMinusPlayers

# print newSndAssistsPlayers
# print Scorers
# newplusMinusPlayers=[]
# for x in range(0,len(plusMinusPlayers)):
#     newplusMinusPlayers=newplusMinusPlayers+plusMinusPlayers[x]
# print newplusMinusPlayers
# # newplusMinusPlayersWin=newplusMinusPlayers[newplusMinusPlayers[1]=='W']
# newplusMinusPlayersWin = [row for row in newplusMinusPlayers if 'W' in row[1]]
# print newplusMinusPlayersWin
# print FstAssistsPlayers
# temp=[]
# for x in range(0,len(newplusMinusPlayersWin)):
#     temp.append(0)
# board = []
# for i in range(3):
#     board.append([])
#     for j in range(1):
#         board[i].append(0)

# temp=newplusMinusPlayersWin[:]
# print "Scorers"
# print Scorers1
# # print Scorers2
# print Scorers3
# print temp
# print FstAssistsPlayers
# print SndAssistsPlayers
# for x in range(0,len(newplusMinusPlayersWin)):
#     print x
#     temp[x].remove(Scorers3[x][1])
#     if FstAssistsPlayers!='unassisted':
#         temp[x].remove(FstAssistsPlayers[x][1])
#         if SndAssistsPlayers!='':
#             temp[x].remove(SndAssistsPlayers[x][1])
# print temp
# print newplusMinusPlayersWin
    
# expe=newplusMinusPlayersWin-FstAssistsPlayers
# print expe
#  Scorers.append(Goal)
#             FstAssistsPlayers.append(FstAssist)
#             SndAssistsPlayers.append(SndAssist)
#             plusMinusPlayers.append(ArrayTemp)

for x in range(0,len(sortedUniqueNewScorers)):
    counter=0
    for y in range(0,len(newScorers)):
        if sortedUniqueNewScorers[x]==newScorers[y]:
            counter=counter+1
    sortedUniqueNewScorers[x].append(counter)

# print sortedUniqueNewScorers

for x in range(0,len(sortedUniqueNewFstAssistsPlayers)):
    counter=0
    for y in range(0,len(newFstAssistsPlayers)):
        if sortedUniqueNewFstAssistsPlayers[x]==newFstAssistsPlayers[y]:
            counter=counter+1
    sortedUniqueNewFstAssistsPlayers[x].append(counter)
# print sortedUniqueNewFstAssistsPlayers

for x in range(0,len(sortedUniqueNewSndAssistsPlayers)):
    counter=0
    for y in range(0,len(newSndAssistsPlayers)):
        if sortedUniqueNewSndAssistsPlayers[x]==newSndAssistsPlayers[y]:
            counter=counter+1
    sortedUniqueNewSndAssistsPlayers[x].append(counter)
    
import csv
# with open('teamAbbreviations.csv', 'rb') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')

# print spamreader


with open('teamAbbreviations.csv',"rU") as csvfileG:
    readCSVG=csv.reader(csvfileG,delimiter=',')
    arrayAllSkaterName=[]
    arrayAllSkaterId=[]

    for row in readCSVG:
        arrayAllSkaterName.append(row[0])
        arrayAllSkaterId.append(row[1])

print arrayAllSkaterName
print arrayAllSkaterId
arrayRequests=[]
def get_poll_xml2(array):
    for i in range(0,6):
        url = "http://%s.nhl.com/club/stats.htm?season=20142015" % array[i]
        arrayRequests.append(requests.get(url).text)
        return requests.get(url).text
arrayZipped=[]
# for i in range(0,31):
#     print i
# #         url = "http://%s.nhl.com/club/stats.htm?season=20142015" % arrayAllSkaterId[i]
# #         soup2=BeautifulSoup(requests.get(url).text.encode('ascii', 'ignore'))
#     url = "http://%s.nhl.com/club/stats.htm?season=20142015" % arrayAllSkaterId[i]
#     soup2=BeautifulSoup(requests.get(url).text.encode('ascii', 'ignore'))
#     resultrwEven= soup2.find_all("tr", attrs={"class":"rwEven"} )
# #         print resultrwEven
#     resultNumberEven=[str(elem.find_all("span", attrs={"class":"sweaterNo"} )[0].string) for elem in resultrwEven]
# #         print resultNumber
#     resultNameEven=[str(elem.find_all("td", attrs={"class":"left"} )[0].string) for elem in resultrwEven]
# #         print resultName
#     resultrwOdd= soup2.find_all("tr", attrs={"class":"rwOdd"} )
#     resultNumberOdd=[str(elem.find_all("span", attrs={"class":"sweaterNo"} )[0].string) for elem in resultrwOdd]
#     resultNameOdd=[str(elem.find_all("td", attrs={"class":"left"} )[0].string) for elem in resultrwOdd]
#     resultNumber=resultNumberOdd+resultNumberEven
#     resultName=resultNameOdd+resultNameEven
#     for x in range(0,len(resultName)):
#         unique3=[]
#         unique3.append(arrayAllSkaterName[i])
#         unique3.append(resultNumber[x])
#         unique3.append(resultName[x])
#         unique3.append(20142015)

#         arrayZipped.append(unique3)
for i in range(0,30):
    print i
#         print arrayAllSkaterId[i]
    url = "http://%s.nhl.com/club/stats.htm?season=20142015" % arrayAllSkaterId[i]
    soup2=BeautifulSoup(requests.get(url).text.encode('ascii', 'ignore'))
    resultrwEven= soup2.find_all("tr", attrs={"class":"rwEven"} )
#         print resultrwEven
#         print resultrwEven
    resultNumberEven=[str(elem.find_all("span", attrs={"class":"sweaterNo"} )[0].string) for elem in resultrwEven if str(elem.find_all("span", attrs={"class":"sweaterNo"}))!='[]']
#         resultNumberEven=[str(elem.find_all("span", attrs={"class":"sweaterNo"} )) for elem in resultrwEven if str(elem.find_all("span", attrs={"class":"sweaterNo"}))!='[]']
#     print resultNumberEven
#         print resultNumber
    resultNameEven=[str(elem.find_all("td", attrs={"class":"left"} )[0].string) for elem in resultrwEven if str(elem.find_all("td", attrs={"class":"left"} ))!='[]']
#         resultNameEven=[str(elem.find_all("td", attrs={"class":"left"} )) for elem in resultrwEven]
#     print resultNameEven
#         print resultName
    resultrwOdd= soup2.find_all("tr", attrs={"class":"rwOdd"} )
    resultNumberOdd=[str(elem.find_all("span", attrs={"class":"sweaterNo"} )[0].string) for elem in resultrwOdd if str(elem.find_all("span", attrs={"class":"sweaterNo"}))!='[]']
    resultNameOdd=[str(elem.find_all("td", attrs={"class":"left"} )[0].string) for elem in resultrwOdd if str(elem.find_all("span", attrs={"class":"sweaterNo"}))!='[]']
    resultNumber=resultNumberOdd+resultNumberEven
    resultName=resultNameOdd+resultNameEven
    for x in range(0,len(resultName)):
        unique3=[]
        unique3.append(arrayAllSkaterName[i])
        unique3.append(resultNumber[x])
        unique3.append(resultName[x])
        unique3.append(20142015)

        arrayZipped.append(unique3)
# get_poll_xml2(arrayAllSkaterId)
# print get_poll_xml2(arrayAllSkaterId)
# print len(arrayAllSkaterId)
# soup2 = BeautifulSoup(get_poll_xml2(arrayAllSkaterId))
# print arrayZipped
print 
# print len(uniquePointsPlayers1)
print 
for i in range(0,len(uniquePointsPlayers1)):
    for y in range(0,len(arrayZipped)):
        if uniquePointsPlayers1[i][:2]==arrayZipped[y][:2]:
            uniquePointsPlayers1[i].insert(0,arrayZipped[y][3])
            uniquePointsPlayers1[i].insert(0,arrayZipped[y][2])
print uniquePointsPlayers1
# goalieMatrix=[]
# for elem in uniquePointsPlayers1:
# # 	for numPlayer in range(uniqueGoalieBeginning[nthTeam], uniqueGoalieEnd[nthTeam]+1):
#     eachPlayer2=[]
#     eachPlayer2.append((
#         elem
#         ))
#     goalieMatrix.append(eachPlayer2)

with open("2014-2015-skater-1-PP.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(uniquePointsPlayers1)
# listi=[20036,20078,20232,20235,20290,20441,20461,20547,20755,20812,20846,20903,21025,21114,21144,21151,21164,21176]


20000
20001
20002
20003
20004
20005
20006
20007
20008
20009
20010
20011
20012
20013
20014
20015
20016
20017
20018
20019
20020
20021
20022
20023
20024
20025
20026
20027
20028
20029
20030
20031
20032
20033
20034
20035
20036
20037
20038
20039
20040
20041
20042
20043
20044
20045
20046
20047
20048
20049
20050
20051
20052
20053
20054
20055
20056
20057
20058
20059
20060
20061
20062
20063
20064
20065
20066
20067
20068
20069
20070
20071
20072
20073
20074
20075
20076
20077
20078
20079
20080
20081
20082
20083
20084
20085
20086
20087
20088
20089
20090
20091
20092
20093
20094
20095
20096
20097
20098
20099
20100
20101
20102
20103
20104
20105
20106
20107
20108
20109
20110
20111
20112
20113
20114
20115
20116
20117
20118
20119
20120
20121
20122
20123
20124
20125
20126
20127
20128
20129
20130
20131
20132
20133
20134
20135
20136
20137
20138
20139
20140
20141
20142
20143
20144
20145
20146
20147
20148
20149
[['DET', '14'], ['DET', '14'], ['L.A', '27']]
[u'EV', u'EV', u'EV']
temp----
[u'DET', 'W', '2', '8',